# Initialization

In [4]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [6]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

### Разбиение на тренировочную и тестовую выборки

In [7]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = 0
for i in users_test.values:
    if i in users_train.values:
        common_users += 1

print(len(users_train), len(users_test), common_users)

428220 123223 120858


### Идентификация холодных пользователей

In [8]:
cold_users = []

for i in users_test.values:
    if i not in users_train.values:
        cold_users.append(i)

print(len(cold_users))

2365


### топ-100 наиболее популярных книг

In [9]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by='popularity_weighted', ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query("avg_rating >= 4")[: 100]

In [10]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 10):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
...,...,...,...,...,...,...,...,...
95,2767052,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",2008,4361,4.413437,19247.0,"{'Young Adult': 30042, 'Fiction': 16754, 'Scie..."
96,18293427,Gabrielle Zevin,The Storied Life of A.J. Fikry,2014,4674,4.092640,19129.0,"{'Fiction': 3795, 'Contemporary': 1100, 'Writi..."
97,3636,Lois Lowry,"The Giver (The Giver, #1)",2006,4667,4.098564,19128.0,"{'Young Adult': 10993, 'Fiction': 9045, 'Class..."
98,18966819,Pierce Brown,"Golden Son (Red Rising, #2)",2015,4361,4.374914,19079.0,"{'Science Fiction': 2613, 'Fantasy': 1372, 'Sc..."


In [11]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]

In [12]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам?
1 - (len(cold_user_recs) / len(cold_users_events_with_recs))

0.8023159636062862

### Подсчет метрик rmse и mae для полученных рекомендаций.

In [13]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2))

0.78 0.62


In [14]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

### Степень разреженности матрицы взаимодействий (U-I-матрицы)

In [15]:
non_empty_cells = events.shape[0]
all_cells = events['user_id'].nunique() * events['item_id'].nunique()
empty_cells = all_cells - non_empty_cells
sparcity = empty_cells / all_cells
sparcity

0.9993451160571009

In [16]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [17]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [18]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8263
MAE:  0.6460
0.826346375350908 0.6460143973270805


In [19]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [20]:
mae_random = accuracy.mae(random_predictions)

MAE:  0.9982


In [21]:
print(0.6460 / 0.9982 * 100)
print(0.9982 / 0.6460 * 100)

64.71648968142657
154.52012383900927


In [22]:
# readers_count = events.groupby(["item_id"]).agg(users=("user_id", "nunique"))
# rare_items = readers_count.query("users < 5")
# events = events[~events['item_id'].isin(rare_items['users'].index.values)]

In [23]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[user_id]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [24]:
rec = get_recommendations_svd(1296647, items, events_train, svd_model)
rec

,item_id,score
0,24812,5.000000
1,8471387,5.000000
2,481749,5.000000
3,30688013,4.996900
4,1108124,4.979711


In [25]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

user_id: 1109853
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
0,Patrick Rothfuss,"The Wise Man's Fear (The Kingkiller Chronicle,...",2015-06-17,2015-08-31,5,"{'Fantasy': 16491, 'Fiction': 2222, 'Fantasy-E..."
1,Andy Weir,The Martian,2014-12-07,2014-12-11,4,"{'Science Fiction': 11966, 'Fiction': 8430}"
2,Brandon Sanderson,"The Way of Kings (The Stormlight Archive, #1)",2015-08-31,2015-10-30,5,"{'Fantasy': 14291, 'Fiction': 1623, 'Fantasy-E..."
3,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",2015-10-30,2016-03-17,5,"{'Fantasy': 8542, 'Fiction': 872, 'Fantasy-Epi..."
4,Ken Follett,"Fall of Giants (The Century Trilogy, #1)",2016-05-12,2016-08-30,4,"{'Historical-Historical Fiction': 4665, 'Ficti..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,30688013,5.000000,Robin Hobb,"Assassin's Fate (The Fitz and the Fool, #3)","{'Fantasy': 1657, 'Fiction': 172, 'Fantasy-Epi..."
1,11221285,5.000000,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
2,19219646,5.000000,Wolfgang Herrndorf,Arbeit und Struktur,"{'Nonfiction': 25, 'European Literature-German..."
3,22037424,4.996202,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
4,2168850,4.995777,"محمد بن إدريس الشافعي, إميل بديع يعقوب",ديوان الإمام الشافعي,"{'Poetry': 93, 'Religion': 15, 'Literature': 1..."


# === Базовые подходы: коллаборативная фильтрация

In [26]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_1868/1044897688.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_1868/1044897688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_1868/1044897688.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [27]:
#  размер матрицы взаимодействий в гигабайтах
events_train["user_id_enc"].nunique() * events_train["item_id_enc"].nunique() / 1024**3

16.54028546065092

In [28]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [29]:
#  размер sparse-матрицы формата CSR  в гигабайтах
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [30]:
# создание и тренировка модели
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/sprint_4/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/sprint_4/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [02:47<00:00,  3.35s/it]


In [31]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

In [32]:
def user_history_and_recommendations(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    user_history = events_train[events_train['user_id'] == user_id]

    history_text = f'''
User id: {user_id} 

    User's history: \n
'''
    for i in range(len(user_history)):
        history_text += '\t'
        history_text += items[items["item_id_enc"] == user_history.iloc[i, :]['item_id_enc']].iloc[0, 1]
        history_text += ': '
        history_text += items[items["item_id_enc"] == user_history.iloc[i, :]['item_id_enc']].iloc[0, 2]
        history_text += '\n'

    print(history_text)

    rec = get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen, n)
    
    recommendation_text = f'''
    User's recommendation:\n
'''
    
    for i in range(n):
        recommendation_text += '\t'
        recommendation_text += items[items["item_id_enc"] == rec.iloc[i, :]['item_id_enc']].iloc[0, 1]
        recommendation_text += ': '
        recommendation_text += items[items["item_id_enc"] == rec.iloc[i, :]['item_id_enc']].iloc[0, 2]
        recommendation_text += ' / '
        if rec.iloc[i, :]['item_id_enc'] in user_history['item_id_enc'].values:
            recommendation_text += 'пользователь прочитал рекоммендованную книгу'
        else:
            recommendation_text += 'пользователь не прочитал рекоммендованную книгу'
        recommendation_text += '\n'
    
    print(recommendation_text)

In [33]:
user_history_and_recommendations(user_item_matrix_train, als_model, 1_000_006, user_encoder, item_encoder, include_seen=True, n=5)


User id: 1000006 

    User's history: 

	Margaret Atwood: The Handmaid's Tale
	Louisa May Alcott: Little Women (Little Women, #1)
	Garth Stein: The Art of Racing in the Rain
	Maria Semple: Where'd You Go, Bernadette
	Roald Dahl, Quentin Blake: Matilda
	Frances Hodgson Burnett: The Secret Garden
	Nina George, Simon Pare: The Little Paris Bookshop
	Aziz Ansari, Eric Klinenberg: Modern Romance
	Kennilworthy Whisp, J.K. Rowling: Quidditch Through the Ages
	Amy Schumer: The Girl with the Lower Back Tattoo
	J.K. Rowling, Mary GrandPré: Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
	Diane Ackerman: The Zookeeper's Wife
	J.K. Rowling, Mary GrandPré: Harry Potter and the Chamber of Secrets (Harry Potter, #2)
	J.K. Rowling, Mary GrandPré: Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)
	Jodi Picoult: Small Great Things
	J.K. Rowling, Mary GrandPré: Harry Potter and the Goblet of Fire (Harry Potter, #4)
	William L. Myers Jr.: A Criminal Defense


    User's recommendation

In [34]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [35]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [36]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [37]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [38]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [39]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [40]:
print(ndcg_at_5_scores.mean())

0.975946709792109


In [41]:
a = als_model.similar_items(itemid=7, N=5, recalculate_item=False, item_users=user_item_matrix_train, filter_items=None, items=None)
for i in range(5):
    print(items[items['item_id_enc'] == a[0][i]]['title'])

2334248    The Hitchhiker's Guide to the Galaxy (Hitchhik...
Name: title, dtype: object
840143    The Restaurant at the End of the Universe (Hit...
Name: title, dtype: object
840144    Life, the Universe and Everything (Hitchhiker'...
Name: title, dtype: object
55118    So Long, and Thanks for All the Fish (Hitchhik...
Name: title, dtype: object
2334250    The Ultimate Hitchhiker's Guide to the Galaxy
Name: title, dtype: object


# === Базовые подходы: контентные рекомендации

In [42]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [43]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v['genre_and_votes']
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [44]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [45]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [46]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [47]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(get_genres(user_items), user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 56)>

In [48]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [49]:
# выведем список жанров, которые предпочитает пользователь

user_genres = get_genres(user_items).copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
0,Fiction,110201,0.195253
14,Classics,96456,0.096687
21,Fantasy,25814,0.074261
15,Science Fiction,14819,0.045902
4,Nonfiction,10179,0.044359


In [50]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(user_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)

In [51]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
861044,J.K. Rowling,"Harry Potter and the Half-Blood Prince (Harry Potter, #6)","{'Fantasy': 46400, 'Young Adult': 15083, 'Fiction': 13083, 'Fantasy-Magic': 3815, 'Childrens': 2..."
2083381,"J.K. Rowling, Mary GrandPré","Harry Potter and the Order of the Phoenix (Harry Potter, #5)","{'Fantasy': 46485, 'Young Adult': 15194, 'Fiction': 13064}"
1584855,"J.K. Rowling, Mary GrandPré","Harry Potter and the Sorcerer's Stone (Harry Potter, #1)","{'Fantasy': 59818, 'Fiction': 17918, 'Young Adult': 17892, 'Fantasy-Magic': 4974, 'Childrens': 4..."
1028676,"J.K. Rowling, Mary GrandPré","Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)","{'Fantasy': 49784, 'Young Adult': 15393, 'Fiction': 14272, 'Fantasy-Magic': 4199, 'Childrens': 3..."
2251360,"J.K. Rowling, Mary GrandPré","Harry Potter and the Goblet of Fire (Harry Potter, #4)","{'Fantasy': 48257, 'Young Adult': 15483, 'Fiction': 13649}"
2335569,"J.K. Rowling, Mary GrandPré","Harry Potter Boxed Set, Books 1-5 (Harry Potter, #1-5)","{'Fantasy': 113, 'Young Adult': 43, 'Fiction': 29, 'Fantasy-Magic': 22, 'Adventure': 9, 'Fantasy..."
2334249,J.K. Rowling,"Harry Potter Collection (Harry Potter, #1-6)","{'Fantasy': 282, 'Fiction': 96, 'Young Adult': 85, 'Fantasy-Magic': 35, 'Adventure': 19, 'Childr..."
2334248,Douglas Adams,"The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)","{'Science Fiction': 13941, 'Fiction': 9853, 'Humor': 5340, 'Fantasy': 4304, 'Classics': 4179, 'H..."
2334250,Douglas Adams,The Ultimate Hitchhiker's Guide to the Galaxy,"{'Science Fiction': 3146, 'Fiction': 2511, 'Humor': 1394, 'Fantasy': 1052, 'Classics': 744}"
1080474,Bill Bryson,A Short History of Nearly Everything,"{'Nonfiction': 7715, 'Science': 5973, 'History': 4648, 'Audiobook': 502}"


# === Базовые подходы: валидация

In [52]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [53]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5)

/tmp/ipykernel_1868/3166051426.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [54]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [55]:
p, r = compute_cls_metrics(events_recs_for_binary_metrics)
print(p)
print(r)

0.007581376853347184
0.014121568795222568


# === Двухстадийный подход: метрики

In [67]:
# расчёт покрытия по объектам
cov_items = als_recommendations['item_id'].nunique() / items['item_id'].nunique()
print(f"{cov_items:.2f}")

0.09


In [69]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train, on=["user_id", "item_id"], how="left")
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by="score", ascending=False)
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
# ваш код здесь #

/tmp/ipykernel_1868/3052700722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["read"] = True


In [70]:
als_recommendations

,user_id,item_id,score,rating_test,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_enc,item_id_enc,read,rank
5783400,1057834,11387515,2.350082,NaN,2012-03-30,2012-04-01,True,3.0,True,2012-03-01,57834.0,29545.0,True,1
16444300,1164443,10964,2.234828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1
13665700,1136657,29056083,2.224506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1
32614600,1326146,9969571,2.193899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1
40560200,1405602,10964,2.181662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8736306,1087363,477,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,96
8736307,1087363,475,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,97
8736308,1087363,463,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,98
8736309,1087363,456,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,99


In [73]:
novelty_5

user_id
1000000    0.2
1000001    0.8
1000002    0.8
1000003    0.0
1000004    0.0
          ... 
1430580    0.8
1430581    0.8
1430582    0.8
1430583    1.0
1430584    0.8
Name: read, Length: 430585, dtype: float64

In [72]:
novelty_5.mean()

0.607333279143491

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков